In [13]:
import numpy as np
import pandas as pd

Loads title.akas.tsv data

In [17]:
title_akas = pd.read_csv('/Users/seankamano/Downloads/title.akas.tsv', delimiter = '\t', encoding = 'utf-8')
print title_akas.head(10)

     titleId  ordering                      title region language  \
0  tt0000001         1  Carmencita - spanyol tánc     HU       \N   
1  tt0000001         2                 Карменсита     RU       \N   
2  tt0000001         3                 Carmencita     US       \N   
3  tt0000001         4                 Carmencita     \N       \N   
4  tt0000002         1     Le clown et ses chiens     \N       \N   
5  tt0000002         2          A bohóc és kutyái     HU       \N   
6  tt0000002         3     Le clown et ses chiens     FR       \N   
7  tt0000002         4     The Clown and His Dogs     US       \N   
8  tt0000002         5         Клоун и его собаки     RU       \N   
9  tt0000003         1            Szegény Pierrot     HU       \N   

         types             attributes isOriginalTitle  
0  imdbDisplay                     \N               0  
1           \N                     \N               0  
2           \N                     \N               0  
3     original  

Need to load title.basics.tsv data to get movies only. Gets tconst for all movies.

In [24]:
title_basics = pd.read_csv('/Users/seankamano/Downloads/title.basics.tsv', delimiter='\t', encoding='utf-8')
title_basics_movie = title_basics[title_basics['titleType'] == 'movie']
uid = title_basics_movie['tconst']

print uid.head(10)

8      tt0000009
145    tt0000147
332    tt0000335
499    tt0000502
571    tt0000574
611    tt0000615
626    tt0000630
669    tt0000675
670    tt0000676
673    tt0000679
Name: tconst, dtype: object


Get movies only from title.akas.tsv by taking uid and comparing them to the tconsts in title.akas.tsv

In [55]:
title_akas_movies = title_akas[title_akas['titleId'].isin(uid.values)]

print title_akas_movies.head(10)

        titleId  ordering                          title region language  \
46    tt0000009         1                     Miss Jerry     \N       \N   
47    tt0000009         2                     Miss Jerry     HU       \N   
48    tt0000009         3                     Miss Jerry     US       \N   
568   tt0000147         1  The Corbett-Fitzsimmons Fight     US       \N   
971   tt0000335         1        Early Christian Martyrs     AU       \N   
972   tt0000335         2          Soldiers of the Cross     AU       \N   
973   tt0000335         3          Soldiers of the Cross     \N       \N   
1447  tt0000502         1                       Bohemios     ES       \N   
1663  tt0000574         1     Kelly bandájának története     HU       \N   
1664  tt0000574         2    The Story of the Kelly Gang     \N       \N   

            types attributes isOriginalTitle  
46       original         \N               1  
47    imdbDisplay         \N               0  
48             \N     

Remove movies without any regional data

In [38]:
title_akas_movies_by_region = title_akas_movies[title_akas_movies['region'] != '\N']
print title_akas_movies_by_region.head(10)

        titleId  ordering                           title region language  \
47    tt0000009         2                      Miss Jerry     HU       \N   
48    tt0000009         3                      Miss Jerry     US       \N   
568   tt0000147         1   The Corbett-Fitzsimmons Fight     US       \N   
971   tt0000335         1         Early Christian Martyrs     AU       \N   
972   tt0000335         2           Soldiers of the Cross     AU       \N   
1447  tt0000502         1                        Bohemios     ES       \N   
1663  tt0000574         1      Kelly bandájának története     HU       \N   
1665  tt0000574         3  Die Geschichte der Kelly Bande     DE       \N   
1666  tt0000574         4     The Story of the Kelly Gang     AU       \N   
1783  tt0000615         1              Robbery Under Arms     AU       \N   

            types attributes isOriginalTitle  
47    imdbDisplay         \N               0  
48             \N         \N               0  
568        

Need to make sense of region codes. Scrape data off of http://www.nationsonline.org/oneworld/country_code_list.htm

In [97]:
import urllib2
from bs4 import BeautifulSoup

soup = BeautifulSoup(urllib2.urlopen('http://www.nationsonline.org/oneworld/country_code_list.htm').read(), "lxml")

tableClassResults = soup.find("table",{"class":"tb86 rowstyle-alternate colstyle-alt no-arrow"})
countries = tableClassResults.find_all('td', {'class':'abs'})

def get_countries():
    for country in countries:
        try: 
            yield country.a.string
        except (AttributeError):
            try:
                yield country.em.string
            except (AttributeError):
                yield country.string
    
list_countries = [str(k for k in get_countries()]

print "Country names: ", list_countries[:10]

country_codes = tableClassResults.find_all('td', {'style':'text-align:center'})

def get_codes():
    for code in country_codes:
        if len(code.string) == 2 or code.string == ' AF':
            yield code.string

list_codes = [i for i in get_codes()]

print "Country codes: ", list_codes[:10]

Country names:  [u'Afghanistan', u'Aland Islands', u'Albania', u'Algeria', u'American Samoa', u'Andorra', u'Angola', u'Anguilla', u'Antarctica  \t  \t  \t', u'Antigua and Barbuda']
Country codes:  [u' AF', u'AX', u'AL', u'DZ', u'AS', u'AD', u'AO', u'AI', u'AQ', u'AG']


Need to change these codes into a form we can work with. Convert into strings and then zip with country names so we have a reference for what each two-character acronym represents.

In [145]:
for i in range(len(list_codes)):
    if str(list_codes[i]) != ' AF':
        list_codes[i] = str(list_codes[i])
    else:
        list_codes[i] = 'AF'

country_dict = dict(zip(list_codes,list_countries))
country_dict

{'AD': u'Andorra',
 'AE': u'United Arab Emirates',
 'AF': u'Afghanistan',
 'AG': u'Antigua and Barbuda',
 'AI': u'Anguilla',
 'AL': u'Albania',
 'AM': u'Armenia',
 'AN': u'Netherlands Antilles',
 'AO': u'Angola',
 'AQ': u'Antarctica  \t  \t  \t',
 'AR': u'Argentina',
 'AS': u'American Samoa',
 'AT': u'Austria',
 'AU': u'Australia',
 'AW': u'Aruba',
 'AX': u'Aland Islands',
 'AZ': u'Azerbaijan',
 'BA': u'Bosnia and Herzegovina',
 'BB': u'Barbados',
 'BD': u'Bangladesh',
 'BE': u'Belgium',
 'BF': u'Burkina Faso',
 'BG': u'Bulgaria',
 'BH': u'Bahrain',
 'BI': u'Burundi',
 'BJ': u'Benin',
 'BL': u'Saint-Barth\xe9lemy',
 'BM': u'Bermuda',
 'BN': u'Brunei Darussalam',
 'BO': u'Bolivia',
 'BR': u'Brazil',
 'BS': u'Bahamas',
 'BT': u'Bhutan',
 'BV': u'Bouvet Island',
 'BW': u'Botswana',
 'BY': u'Belarus',
 'BZ': u'Belize',
 'CA': u'Canada',
 'CC': u'Cocos (Keeling) Islands',
 'CD': u'Congo',
 'CF': u'Central African Republic',
 'CG': u'Congo',
 'CH': u'Switzerland',
 'CI': u"C\xf4te d'Ivoire",

Now have a more recognizable form for each country code. Now to find distribution of number of movies across these countries as well as average rating of movies by coutry.

In [169]:
movies_per_country = dict(zip(list_codes, [0 for i in range(len(list_codes))]))
avg_rating_per_country = dict(zip(list_codes, [0 for i in range(len(list_codes))]))
for code in title_akas_movies_by_region['region']:
    try:
        movies_per_country[str(code)] += 1
    except(KeyError):
        continue

movies_per_country        

{'AD': 11,
 'AE': 129,
 'AF': 77,
 'AG': 11,
 'AI': 1,
 'AL': 954,
 'AM': 184,
 'AN': 7,
 'AO': 36,
 'AQ': 4,
 'AR': 17519,
 'AS': 2,
 'AT': 11654,
 'AU': 8740,
 'AW': 7,
 'AX': 0,
 'AZ': 424,
 'BA': 406,
 'BB': 16,
 'BD': 1133,
 'BE': 11588,
 'BF': 92,
 'BG': 13407,
 'BH': 24,
 'BI': 5,
 'BJ': 17,
 'BL': 0,
 'BM': 10,
 'BN': 4,
 'BO': 196,
 'BR': 41831,
 'BS': 41,
 'BT': 29,
 'BV': 0,
 'BW': 1,
 'BY': 119,
 'BZ': 8,
 'CA': 22046,
 'CC': 0,
 'CD': 14,
 'CF': 2,
 'CG': 20,
 'CH': 2595,
 'CI': 31,
 'CK': 0,
 'CL': 3913,
 'CM': 78,
 'CN': 4808,
 'CO': 2143,
 'CR': 132,
 'CU': 473,
 'CV': 3,
 'CX': 0,
 'CY': 98,
 'CZ': 3798,
 'DE': 34688,
 'DJ': 1,
 'DK': 22721,
 'DM': 3,
 'DO': 438,
 'DZ': 185,
 'EC': 211,
 'EE': 2798,
 'EG': 2805,
 'EH': 1,
 'ER': 7,
 'ES': 38935,
 'ET': 41,
 'FI': 30604,
 'FJ': 13,
 'FK': 0,
 'FM': 0,
 'FO': 4,
 'FR': 48851,
 'GA': 18,
 'GB': 29366,
 'GD': 1,
 'GE': 583,
 'GF': 0,
 'GG': 0,
 'GH': 74,
 'GI': 1,
 'GL': 22,
 'GM': 2,
 'GN': 8,
 'GP': 11,
 'GQ': 1,
 'GR': 

Average ratings of movies per country: